In [12]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from collections import Counter
import requests
import string

In [2]:
full_dat = pd.read_excel('CleanedUp.xlsx')

In [3]:
full_dat.head()

,TYPE,ORGANIZATION,ADDRESS,CITY,STATE,ZIP,OFFICE PHONE,WEBSITE,DESCRIPTION/MISSION,KEYWORDS,AGE BRACKET
0,Religious,Affirmation Houston Chapter (LDS),3805 W Alabama Street #6102,Houston,TX,77027.0,7134879854,www.affirmation.org,As members and friends of the gay and lesbian ...,"Religious, Christian, LGBT",NaN
1,Education & Services,"AIDS Foundation Houston, Inc.",6260 Westpark Dr. Suite 100,Houston,TX,77027.0,(713) 623-6796,www.aidshelp.org,AIDS foundation lead the innovative efforts in...,"HIV/AIDS, health, housing, Medical Testing, Food",NaN
2,Social,Alpha Lambda Zeta Fraternity,NaN,NaN,NaN,NaN,(713) 449-4026,www.alphalambdazeta.com,"Alpha Lambda Zeta (ΑΛΖ), an LGBT Greek-Lettere...","Lesbian, African-American, Black, POC",NaN
3,Education & Services,South Texas College of Law - AMICUS,1303 San Jacinto MB #1,Houston,TX,77002.0,(713) 659-8040,http://www.stcl.edu/student-services/student-o...,"Amicus educates, advises and informs its membe...","LGBT, education, school, GSA",NaN
4,History,Area Rainbow Collective History - Houston,NaN,NaN,NaN,NaN,NaN,http://houstonarch.pbworks.com/w/page/19526143...,ARCH was established as a coalition of Houston...,"History, LGBT, arts",NaN


In [4]:
full_dat['description_reduced'] = full_dat['DESCRIPTION/MISSION'].fillna("na").apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopwords.words('english')]))

In [5]:
words = []
for item in list(full_dat['description_reduced']):
    words += [x for x in item.split()]
    
word_count = Counter(words)

In [6]:
high_hits = {x: y for x, y in word_count.items() if y > 4}

In [7]:
high_hits

{'members': 7,
 'gay': 13,
 'community,': 6,
 'within': 5,
 'help': 10,
 'HIV': 6,
 'HIV/AIDS': 7,
 'Houston,': 8,
 'Inc.': 7,
 'community': 36,
 'individuals': 7,
 'lives': 5,
 'without': 5,
 'LGBT': 19,
 'promote': 9,
 'positive': 5,
 'political': 6,
 'gay,': 10,
 'lesbian,': 10,
 'bisexual': 5,
 'GLBT': 5,
 'providing': 10,
 'also': 6,
 'Houston': 39,
 'social': 21,
 'educational': 7,
 'bisexual,': 7,
 'transgender,': 5,
 'queer': 5,
 '&': 6,
 'group': 14,
 'dedicated': 12,
 'Gay': 9,
 'regardless': 5,
 'sexual': 5,
 'advance': 5,
 'diverse': 5,
 'love': 5,
 'committed': 5,
 'among': 7,
 'adults': 7,
 'like': 5,
 'women': 7,
 'one': 5,
 'services': 15,
 '18': 6,
 'health': 10,
 'organization': 20,
 'quality': 5,
 'rights': 7,
 'community.': 6,
 'Center': 10,
 'mission': 14,
 'LGBTQ': 13,
 'promoting': 5,
 'men': 20,
 'support': 9,
 'Texas': 12,
 'provide': 19,
 'foster': 5,
 'safe': 11,
 'transgender': 6,
 'ages': 6,
 'support,': 6,
 'non-profit': 6,
 'serving': 5,
 'area.': 5,
 'li

In [8]:
def get_synonyms(word):
    base_url = "http://thesaurus.altervista.org/thesaurus/v1?key=XY4VGGI2PmNP9HPdaBMY&language=en_US&output=json&word="
    r = requests.get(base_url + word)
    response = r.json()
    clean = [word]
    try:
        combine = [x['list']['synonyms'] for x in response['response']]
        for item in combine:
            for word in item.split('|'):
                clean += [word.split(' (')[0]]
    except:
        pass
    return list(set(clean))


def flatten(lst):
    res = []
    for item in lst:
        if type(item) == list:
            for subitem in item:
                res += [subitem]
        else:
            res += [item]
    return res
            

full_dat['search_terms'] = full_dat['KEYWORDS'].apply(lambda x: ','.join(flatten([get_synonyms(word) for word in x.split(', ')])))


In [9]:
full_dat['search_terms']

0      Religious,scrupulous,churchlike,god-fearing,sp...
1      HIV/AIDS,well-being,eudaimonia,status,health,u...
2      tribade,gay woman,Lesbian,homo,gay,Greek,Helle...
3      LGBT,Department of Education,instruction,activ...
4      arts,humanistic discipline,continuum,record,ch...
5      dweller,denizen,oriental,inhabitant,person of ...
6      tribade,gay woman,Lesbian,homo,gay,Greek,Helle...
7      HIV/AIDS,well-being,eudaimonia,status,health,u...
8      colourful,homosexual,individual,somebody,festi...
9      Arts,field of study,discipline,humanistic disc...
10     colourful,homosexual,individual,somebody,festi...
11     Women,field of study,discipline,arts,humanisti...
12     HIV/AIDS,assist,aid,assistance,financial aid,h...
13     unpaid,military volunteer,military man,service...
14     Department of Education,instruction,activity,u...
15     LGBT,society,club,gild,guild,association,order...
16     LGBT,history,arts,humanistic discipline,contin...
17     frat,Fraternity,society,

In [10]:
all_search_terms = []
for item in list(full_dat['search_terms']):
    for word in item.split(", "):   
        all_search_terms += [word]
        
word_counts = Counter(all_search_terms)

{k: v for k, v in word_counts.items() if v > 4}

{'Sports,aggroup,squad,team,animal group,team up,unit,group,social unit,LGBT': 6}

In [13]:
test_message = "hi, vtwmk wtknvltkr looking for a safe space"

def get_hits(msg):
    words = ''.join([x for x in test_message if x not in string.punctuation]).split()
    hits = []
    for item in words:
        temp_res = full_dat[full_dat['search_terms'].str.contains(item)]
    return temp_res
    

res = get_hits(test_message)

In [14]:
# rank search terms by number of entries with the number of occurences (i.e. rare words weighted more heavily)
res['search_terms'][139]

'LGBT,dispossessed,homeless,poor,unfortunate,homeless person,poor people,roofless,unfortunate person,stateless,unsettled,food,food for thought,content,nutrient,substance,cognitive content,solid,intellectual nourishment,mental object,solid food,matter,showers,clothing,wearable,vesture,consumer goods,wear,article of clothing,habiliment,covering,cyberspace,net,computer network,internet'

In [15]:
org_html = full_dat[['TYPE', 'ORGANIZATION', 'DESCRIPTION/MISSION', 'WEBSITE', 'KEYWORDS']].to_html()

In [16]:
org_mapping = full_dat[['search_terms']].to_json()

In [17]:
rows = org_html.split("<tr>")

In [18]:
i = 1
while i < len(rows):
    id = rows[i].split("</th>")[0].split(">")[-1]
    rows[i] = f"<tr id={id}>" + rows[i]
    i += 1
    
org_html = "".join(rows)

lines = org_html.split("\n")
i = 9
while i < len(lines):  
     if "<th>" in lines[i]:
         lines.remove(lines[i])
     if "www" in lines[i] or "http" in lines[i] and "<td>" in lines[i]:
         website = lines[i].split(">")[1].split("<")[0]
         lines[i] = f"<td><a href='{website}'>{website}</a></td>"
     i += 1

org_html = '\n'.join(lines)

In [19]:
with open("oSTEM2018_datatable.html", "w") as f:
    f.write(org_html)

In [20]:
temp = full_dat[['search_terms']]

mapp = {}
i = 0
while i < len(temp):
    search_terms = temp.iloc[i].at['search_terms']
    if search_terms is not None:
        search_terms = search_terms.split(",")
        for item in search_terms:
            if item in mapp.keys():
                mapp[item.lower()] += [i]
            else:
                mapp[item.lower()] = [i]
    i += 1
    
mapp

{'religious': [128],
 'scrupulous': [0, 18, 49, 51, 52, 86, 93, 128],
 'churchlike': [0, 18, 49, 51, 52, 86, 93, 128],
 'god-fearing': [0, 18, 49, 51, 52, 86, 93, 128],
 'spiritual': [0, 18, 49, 51, 52, 86, 93, 128],
 'interfaith': [0, 18, 49, 51, 52, 86, 93, 128],
 'irreligious': [0, 18, 49, 51, 52, 86, 93, 128],
 'sacred': [0, 18, 49, 51, 52, 86, 93, 128],
 'churchly': [0, 18, 49, 51, 52, 86, 93, 128],
 'religious person': [0,
  0,
  18,
  18,
  49,
  49,
  51,
  51,
  52,
  86,
  86,
  93,
  93,
  128,
  128],
 'devout': [0, 18, 49, 51, 52, 86, 93, 128],
 'pious': [0, 18, 49, 51, 52, 86, 93, 128],
 'churchgoing': [0, 18, 49, 51, 52, 86, 93, 128],
 'faith': [0, 18, 51, 52, 86, 93, 128],
 'christly': [0, 18, 51, 86, 93, 128],
 'unchristian': [0, 18, 51, 86, 93, 128],
 'christianly': [0, 18, 51, 86, 93, 128],
 'religious belief': [0, 18, 51, 86, 93, 128],
 'religion': [0, 18, 51, 52, 86, 93, 128],
 'christian': [128],
 'christlike': [0, 18, 51, 86, 93, 128],
 'lgbt': [140],
 'hiv/aids'

In [21]:
import json

json.dump(mapp, open("oSTEM_search_mapping.json", "w"))